------------- FUNCTION EXPLANATIONS -------------<br/>
break  : stop the loop to the first iteration <br/>
split : it'll take any texts and make 2 blocks with it (before and after)<br/>
df : shortcut in order to access to the excel file (such pd for pandas)<br/>
loc[n] : locate the first line

------------- OBJECTIVES OF THE SESSIONS -------------<br/>
1- find the iso codes<br/>
2- put it inside the code<br/>
3- Displays the last gc tax for each country<br/>

In [4]:
import pandas as pd
import requests as rq

In [5]:
df = pd.read_excel('Data-sheet-STUDENTS.xlsx',header=2,index_col=0)

In [6]:
iso_codes = {}    #create a table that will contain all the iso_codes of each countries

In [7]:
for index, name in df['Country (according to CN Gov):'].items():  #create the loop to only have the names of the countries
    if name in iso_codes:
        continue    #means that if the name already exists then it'll pass the step

    cleaned_name = name.split('(')[0].strip()    #remove the error of France (Not BRI) / Thailiand / USA...
    if cleaned_name == 'Thailiand':
        cleaned_name = 'Thailand'
    if cleaned_name == 'The United Arab Emirates':
        cleaned_name = 'United Arab Emirates'
    
        
    url = 'https://restcountries.eu/rest/v2/name/' + cleaned_name   #don't forget to change into cleaned_name
    r = rq.get(url)  #create the fonction request to obtain the isocodes
    if r.status_code == 200:
        data = r.json() #check the status code if 200 it's success
        country_code = data[0]['alpha3Code']   #enter inside the table and go to the first column and then to the parameter
        if country_code == 'IOT':
            country_code = 'IND'
        iso_codes[name] = country_code   #put the code inside the table
    else:
        print('ERROR',r.status_code,r.text)
        print(name)

In [8]:
df['isocode'] = df['Country (according to CN Gov):'].apply(lambda x: iso_codes[x])

In [9]:
df[['isocode','Country (according to CN Gov):']]

,isocode,Country (according to CN Gov):
1,AFG,Afghanistan
2,ALB,Albania
3,ARM,Armenia
4,AZE,Azerbaijan
5,BHR,Bahrain
6,BGD,Bangladesh
7,BLR,Belarus
8,BTN,Bhutan
9,BIH,Bosnia and Herzegovina
10,BRN,Brunei


In [10]:
tax_values={}

In [11]:
for index, iso in df['isocode'].items():
    
    url_tax = 'http://api.worldbank.org/v2/country/'+iso+'/indicator/GC.TAX.IMPT.ZS?format=json&MRV=1' 
    print(url_tax)
    r = rq.get(url_tax) 
    if r.status_code == 200:
        data = r.json() 
        if data[1] is None:    #prevent the problem of countries without any tax value
            tax_country = 0
        else:
            tax_country = data[1][0]['value']  
        tax_values[iso] = tax_country
    else:
        print('ERROR',r.status_code,r.text)
        print(iso)     

http://api.worldbank.org/v2/country/AFG/indicator/GC.TAX.IMPT.ZS?format=json&MRV=1
http://api.worldbank.org/v2/country/ALB/indicator/GC.TAX.IMPT.ZS?format=json&MRV=1
http://api.worldbank.org/v2/country/ARM/indicator/GC.TAX.IMPT.ZS?format=json&MRV=1
http://api.worldbank.org/v2/country/AZE/indicator/GC.TAX.IMPT.ZS?format=json&MRV=1
http://api.worldbank.org/v2/country/BHR/indicator/GC.TAX.IMPT.ZS?format=json&MRV=1
http://api.worldbank.org/v2/country/BGD/indicator/GC.TAX.IMPT.ZS?format=json&MRV=1
http://api.worldbank.org/v2/country/BLR/indicator/GC.TAX.IMPT.ZS?format=json&MRV=1
http://api.worldbank.org/v2/country/BTN/indicator/GC.TAX.IMPT.ZS?format=json&MRV=1
http://api.worldbank.org/v2/country/BIH/indicator/GC.TAX.IMPT.ZS?format=json&MRV=1
http://api.worldbank.org/v2/country/BRN/indicator/GC.TAX.IMPT.ZS?format=json&MRV=1
http://api.worldbank.org/v2/country/BGR/indicator/GC.TAX.IMPT.ZS?format=json&MRV=1
http://api.worldbank.org/v2/country/KHM/indicator/GC.TAX.IMPT.ZS?format=json&MRV=1
http

In [12]:
#create the first table with all the tax without making the link with the real column
df['Tax Values'] = df['isocode'].apply(lambda x: tax_values[x])
df['Tax Values']
#print(tax_values)

1     35.203141
2      2.216459
3      6.275395
4      7.571847
5     27.386091
6     28.923008
7      8.200619
8      2.840501
9      0.000155
10     0.000000
11     0.068860
12    13.542016
13     3.894341
14    -0.037631
15     0.819567
16     4.779919
17     6.849002
18     0.039429
19    -0.023245
20     0.757021
21     0.021249
22    -0.000145
23    12.808273
24     2.568975
25    20.696450
26    14.105211
27     0.962753
28     7.006193
29     5.299519
30    66.806723
        ...    
39     3.423233
40    13.502166
41     0.000000
42     9.003222
43    17.770038
44     0.000000
45     0.000000
46    20.357608
47     0.128546
48     0.000000
49     0.035332
50     6.217626
51    23.017305
52     5.062219
53     0.071508
54     0.000066
55     0.808280
56    22.054891
57     0.000000
58    15.943747
59     3.952757
60     0.381747
61     2.504523
62     0.000000
63     0.047747
64     2.642594
65     3.997681
66     5.186860
67     0.000000
68     0.000000
Name: Tax Values, Length

In [13]:
df['IMF WB\nCustoms and other import duties (% of tax revenue) 2016']

1        NaN
2        NaN
3        NaN
4        NaN
5        NaN
6        NaN
7        NaN
8        NaN
9        NaN
10       NaN
11       NaN
12    15.785
13       NaN
14       NaN
15       NaN
16       NaN
17       NaN
18       NaN
19       NaN
20       NaN
21       NaN
22       NaN
23       NaN
24       NaN
25       NaN
26       NaN
27       NaN
28       NaN
29       NaN
30       NaN
       ...  
39       NaN
40       NaN
41       NaN
42       NaN
43       NaN
44       NaN
45       NaN
46       NaN
47       NaN
48       NaN
49       NaN
50       NaN
51       NaN
52       NaN
53       NaN
54       NaN
55       NaN
56       NaN
57       NaN
58       NaN
59       NaN
60       NaN
61       NaN
62       NaN
63       NaN
64       NaN
65       NaN
66       NaN
67       NaN
68       NaN
Name: IMF WB\nCustoms and other import duties (% of tax revenue) 2016, Length: 68, dtype: float64

In [14]:
#make a loop that will look if the cell is empty or not
#if it's empty it'll had the value from the new table if it's not it'll let it like it is

for index, tax in df['IMF WB\nCustoms and other import duties (% of tax revenue) 2016'].items():
    if pd.isnull(df.loc[index,'IMF WB\nCustoms and other import duties (% of tax revenue) 2016']):
        df.loc[index,'IMF WB\nCustoms and other import duties (% of tax revenue) 2016'] = df.at[index,'Tax Values']

In [15]:
df[['isocode','IMF WB\nCustoms and other import duties (% of tax revenue) 2016']]

,isocode,IMF WB Customs and other import duties (% of tax revenue) 2016
1,AFG,35.203141
2,ALB,2.216459
3,ARM,6.275395
4,AZE,7.571847
5,BHR,27.386091
6,BGD,28.923008
7,BLR,8.200619
8,BTN,2.840501
9,BIH,0.000155
10,BRN,0.000000


------------------------------------------------------------------------------------------------------------------------        
                                            THIRD PART
                                            
        - Create a table that contains all the URL needed
        - we select the concern row
        - create a loop for each column of the row 
        - if the url of the column contains 'wordbank'
        - if yes it'll put the url inside a table
        - now we have to isolate the last part of the URL after indicator/
        - once we did it we can create a table that contains all the last part and make a loop

        - THE PROBLEM COMES FROM THE URL THAT WE EXTRACT HERE WE NEED TO FIND A WAY TO DON'T HAVE '...'

In [207]:
list_url = {}

In [208]:
#df_link = pd.read_excel('Data-sheet-STUDENTS.xlsx',header=None,skiprows=1,nrows=1,usecols="D:AR")
df_link = pd.read_excel('Data-sheet-STUDENTS.xlsx',header=1,nrows=1,usecols="D:AR")
df_link = df_link.apply(str).str.replace('NaN', 'No link') 


In [209]:
for index, urlName in df_link.items():
    if 'data.worldbank' in urlName:
        extracted_url = urlName.split('/')[4].split(',')[0]
        print(extracted_url)
        list_url[extracted_url] = extracted_url

GC.TAX.IMPT.ZS
FB.AST.NPER.ZS
IS.RRS.GOOD.MT.K6
IS.RRS.GOOD.MT.K6.1
NE.CON.PRVT.PP.CD
GB.XPD.RSDV.GD.ZS?locations=MY-KH
SP.POP.SCIE.RD.P6
BX.KLT.DINV.WD.GD.ZS
SH.XPD.CHEX.PP.CD
GB.XPD.RSDV.GD.ZS
IP.TMK.RSCT
IP.PAT.RESD


In [210]:
df_link['DATA Worldbank'] = list_url
df_link['DATA Worldbank']

{'GC.TAX.IMPT.ZS': 'GC.TAX.IMPT.ZS',
 'FB.AST.NPER.ZS': 'FB.AST.NPER.ZS',
 'IS.RRS.GOOD.MT.K6': 'IS.RRS.GOOD.MT.K6',
 'IS.RRS.GOOD.MT.K6.1': 'IS.RRS.GOOD.MT.K6.1',
 'NE.CON.PRVT.PP.CD': 'NE.CON.PRVT.PP.CD',
 'GB.XPD.RSDV.GD.ZS?locations=MY-KH': 'GB.XPD.RSDV.GD.ZS?locations=MY-KH',
 'SP.POP.SCIE.RD.P6': 'SP.POP.SCIE.RD.P6',
 'BX.KLT.DINV.WD.GD.ZS': 'BX.KLT.DINV.WD.GD.ZS',
 'SH.XPD.CHEX.PP.CD': 'SH.XPD.CHEX.PP.CD',
 'GB.XPD.RSDV.GD.ZS': 'GB.XPD.RSDV.GD.ZS',
 'IP.TMK.RSCT': 'IP.TMK.RSCT',
 'IP.PAT.RESD': 'IP.PAT.RESD'}

In [211]:
tab=[]

In [214]:
#Get the indicator for all the countries
#ATTENTION ! We have to create a table each time that the loop iso did a loop
#create a table like that :       iso   /   tax_value(src)
#                                 AFG   /    2.57845

for index_src, src in df_link['DATA Worldbank'].items():
    print(index_src)
    if index_src==src:
        for index, iso in df['isocode'].items():
            #print(index)
            url_src = 'http://api.worldbank.org/v2/country/'+iso+'/indicator/'+src+'?format=json&MRV=1' 
            print(iso)
            r = rq.get(url_src)
            if 'GB.XPD.RSDV.GD.ZS?locations=MY-KH' in url_src:
                continue
            if 'IS.RRS.GOOD.MT.K6.1' in url_src:
                continue
            if r.status_code == 200:
                data = r.json() 
                if data[1] is None:    
                    tax_country = 0
                else:
                    tax_country = data[1][0]['value'] 
                    #tab[iso] = {'Source': src, 'Tax':tax_country}
                    tab.append([iso,src,tax_country])
                    #print(value_tax)
            else:
                print('no')
            

GC.TAX.IMPT.ZS
AFG
ALB
ARM
AZE
BHR
BGD
BLR
BTN
BIH
BRN
BGR
KHM
CHN
HRV
CZE
TLS
EGY
EST
FRA
GEO
DEU
HUN
IND
IDN
IRN
IRQ
ISR
JOR
KAZ
KWT
KGZ
LAO
LVA
LBN
LTU
MKD
MYS
MDV
MDA
MNG
MNE
MMR
NPL
OMN
PAK
PHL
POL
QAT
ROU
RUS
SAU
SRB
SGP
SVK
SVN
LKA
SYR
TJK
THA
ARE
TUR
TKM
GBR
USA
UKR
UZB
VNM
YEM
FB.AST.NPER.ZS
AFG
ALB
ARM
AZE
BHR
BGD
BLR
BTN
BIH
BRN
BGR
KHM
CHN
HRV
CZE
TLS
EGY
EST
FRA
GEO
DEU
HUN
IND
IDN
IRN
IRQ
ISR
JOR
KAZ
KWT
KGZ
LAO
LVA
LBN
LTU
MKD
MYS
MDV
MDA
MNG
MNE
MMR
NPL
OMN
PAK
PHL
POL
QAT
ROU
RUS
SAU
SRB
SGP
SVK
SVN
LKA
SYR
TJK
THA
ARE
TUR
TKM
GBR
USA
UKR
UZB
VNM
YEM
IS.RRS.GOOD.MT.K6
AFG
ALB
ARM
AZE
BHR
BGD
BLR
BTN
BIH
BRN
BGR
KHM
CHN
HRV
CZE
TLS
EGY
EST
FRA
GEO
DEU
HUN
IND
IDN
IRN
IRQ
ISR
JOR
KAZ
KWT
KGZ
LAO
LVA
LBN
LTU
MKD
MYS
MDV
MDA
MNG
MNE
MMR
NPL
OMN
PAK
PHL
POL
QAT
ROU
RUS
SAU
SRB
SGP
SVK
SVN
LKA
SYR
TJK
THA
ARE
TUR
TKM
GBR
USA
UKR
UZB
VNM
YEM
IS.RRS.GOOD.MT.K6.1
AFG
ALB
ARM
AZE
BHR
BGD
BLR
BTN
BIH
BRN
BGR
KHM
CHN
HRV
CZE
TLS
EGY
EST
FRA
GEO
DEU
HUN
IND
IDN
IRN
IRQ
ISR
JOR
KAZ


In [215]:
df_test = pd.DataFrame(tab)
print(df_test)

       0               1              2
0    AFG  GC.TAX.IMPT.ZS      35.203141
1    ALB  GC.TAX.IMPT.ZS       2.216459
2    ARM  GC.TAX.IMPT.ZS       6.275395
3    AZE  GC.TAX.IMPT.ZS       7.571847
4    BHR  GC.TAX.IMPT.ZS      27.386091
5    BGD  GC.TAX.IMPT.ZS      28.923008
6    BLR  GC.TAX.IMPT.ZS       8.200619
7    BTN  GC.TAX.IMPT.ZS       2.840501
8    BIH  GC.TAX.IMPT.ZS       0.000155
9    BGR  GC.TAX.IMPT.ZS       0.068860
10   KHM  GC.TAX.IMPT.ZS      13.542016
11   CHN  GC.TAX.IMPT.ZS       3.894341
12   HRV  GC.TAX.IMPT.ZS      -0.037631
13   CZE  GC.TAX.IMPT.ZS       0.819567
14   TLS  GC.TAX.IMPT.ZS       4.779919
15   EGY  GC.TAX.IMPT.ZS       6.849002
16   EST  GC.TAX.IMPT.ZS       0.039429
17   FRA  GC.TAX.IMPT.ZS      -0.023245
18   GEO  GC.TAX.IMPT.ZS       0.757021
19   DEU  GC.TAX.IMPT.ZS       0.021249
20   HUN  GC.TAX.IMPT.ZS      -0.000145
21   IND  GC.TAX.IMPT.ZS      12.808273
22   IDN  GC.TAX.IMPT.ZS       2.568975
23   IRN  GC.TAX.IMPT.ZS      20.696450


In [219]:
if 'IP.PAT.RESD' in tab:
    print('yes')
else:
    print('no')

no
